# Notebook Goal

The goal of this notebook is simply to check the format of the pageviews data.  

The next notebook will process the data and save it in a parquet file.  


Note this comment:
```
KNOWN ISSUE: rows without Page IDs have only 5 columns, while rows with Page IDs have 6. We are applying a fix which will take some time.
Details on data segments
```



Otherwise this is the format we expect:

**Format**
```
wiki code (subproject.project)
article title
page id
daily total
hourly counts
Hourly counts can be deciphered as follows:
```

**Decoding hourly counts**
```
Hour:
from 0 to 23, written as 0 = A, 1 = B ... 22 = W, 23 = X
```

In [1]:
"""
Cat the first 10 lines of the pageviews data to see the format
"""

import os

DATADIR = os.path.abspath("../../data/wikipedia_meta/")


test_file = os.path.join(DATADIR, "pageviews-202501-user.bz2")

!bzcat {test_file} | head -n 10 > test_data.txt

!cat test_data.txt



bzcat: I/O or other error, bailing out.  Possible reason follows.
bzcat: Broken pipe
	Input file = /Users/einar/git/hafsteinn/together_rag/data/wikipedia_meta/pageviews-202501-user.bz2, output file = (stdout)
aa.wikibooks - null mobile-web 11 G2U1V8
aa.wikibooks - null desktop 6 A2B1M1V1\1
aa.wikibooks Category:Candidates_for_speedy_deletion null desktop 6 U3\3
aa.wikibooks File:Baku4_meetup11.JPG null mobile-web 1 U1
aa.wikibooks File:Baku5_meetup9.JPG null mobile-web 1 V1
aa.wikibooks File:Books_from_the_Biodiversity_Heritage_Library_(IA_tabularviewexte00Leac).pdf null mobile-web 1 W1
aa.wikibooks File:Broom_icon.svg null desktop 1 C1
aa.wikibooks File:Bättlihorn_-_Bettmeralp_(valo139).JPEG null desktop 1 Q1
aa.wikibooks File:CapX_-_Illustration_8.svg null mobile-web 1 \1
aa.wikibooks File:Commons-logo.svg null mobile-web 3 G3


In [5]:
import pandas as pd
keep_data = []
with open("test_data.txt", "r") as f:
    for line in f:
        l = line.strip().split()
        if len(l) == 5:
            continue
        elif len(l) == 6:
            l = [w.strip() for w in l]
            keep_data.append(l)
        else:
            raise ValueError(f"Line has {len(l)} columns: {l}")

len(keep_data)

df = pd.DataFrame(keep_data, columns=['wiki_code', 'article_title', 'page_id', 'access_method', 'daily_total', 'hourly_counts'])

df

,wiki_code,article_title,page_id,access_method,daily_total,hourly_counts
0,aa.wikibooks,-,null,mobile-web,11,G2U1V8
1,aa.wikibooks,-,null,desktop,6,A2B1M1V1\1
2,aa.wikibooks,Category:Candidates_for_speedy_deletion,null,desktop,6,U3\3
3,aa.wikibooks,File:Baku4_meetup11.JPG,null,mobile-web,1,U1
4,aa.wikibooks,File:Baku5_meetup9.JPG,null,mobile-web,1,V1
5,aa.wikibooks,File:Books_from_the_Biodiversity_Heritage_Libr...,null,mobile-web,1,W1
6,aa.wikibooks,File:Broom_icon.svg,null,desktop,1,C1
7,aa.wikibooks,File:Bättlihorn_-_Bettmeralp_(valo139).JPEG,null,desktop,1,Q1
8,aa.wikibooks,File:CapX_-_Illustration_8.svg,null,mobile-web,1,\1
9,aa.wikibooks,File:Commons-logo.svg,null,mobile-web,3,G3
